In [10]:
import requests
import json
from pandas import json_normalize
import pandas as pd

# Count the number of HPO ID
### total 16480

In [4]:
from owlready2 import *
onto = get_ontology("http://purl.obolibrary.org/obo/hp.owl").load()
obo = onto.get_namespace("http://purl.obolibrary.org/obo/")
ontology_classes = obo.HP_0000001.descendants()

i = 0
for current_class in ontology_classes:
    hp_id = current_class.name.replace('_', ':')
    i = i + 1
print(str(i))

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/BFO_0000023
  http://purl.obolibrary.org/obo/CHEBI_50906

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/CHEBI_36080
  http://purl.obolibrary.org/obo/PR_000000001



16480


# Find descendants of Muscular dystrophy

In [6]:
from pyhpo import Ontology
_ = Ontology()
term = Ontology.get_hpo_object('HP:0003560')
for p in term.children:
    print(p)

HP:0007081 | Late-onset muscular dystrophy
HP:0006785 | Limb-girdle muscular dystrophy
HP:0003741 | Congenital muscular dystrophy
HP:0008970 | Scapulohumeral muscular dystrophy


# Co-occurence & Jaccard Index

In [7]:
query = {'dataset_id': 2, 'concept_id_1':'90003560', 'concept_id_2': ['90006785; 90007081; 90008970; 90003741']}
response = requests.get('https://rare.cohd.io/api/frequencies/pairedConceptFreq', params=query)

info = response.json()
df = json_normalize(info['results'])
df['Jaccard'] = df['concept_pair_count']/(df['concept_count_1']+df['concept_count_2']-df['concept_pair_count']) 
df_sorted = df.sort_values(by=['concept_pair_count'], ascending= False)
n_df = df_sorted.loc[:, ("concept_name_2", "concept_pair_count", "Jaccard")]
n_df

,concept_name_2,concept_pair_count,Jaccard
0,Limb-girdle muscular dystrophy,315,0.191257
1,Congenital muscular dystrophy,174,0.106552
2,Late-onset muscular dystrophy,22,0.013631


# With All HPO Terms

In [7]:
o_list = [] 
for current_class in ontology_classes:
    hp_id = current_class.name.replace('_', ':')
    o_list.append(hp_id)

16480

In [14]:
o_100 = o_list[0:100]
s = ",".join(o_100).replace(",", ";")
para = {'q': s}
response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params = para).json()
df_z = json_normalize(response['results'])
list_z = df_z['concept_id'].values.tolist()
list_z = list(map(str, list_z))
str_z = ",".join(list_z).replace(",", ";")
query = {'dataset_id': 2, 'concept_id_1':'90003560', 'concept_id_2': str_z}
info = requests.get('https://rare.cohd.io/api/frequencies/pairedConceptFreq', params=query).json()
result = json_normalize(info['results'])

In [12]:
i = 101
while i < 16400: 
    o_100 = o_list[i:i+99]
    s = ",".join(o_100).replace(",", ";")
    para = {'q': s}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params = para).json()
    df_z = json_normalize(response['results'])
    list_z = df_z['concept_id'].values.tolist()
    list_z = list(map(str, list_z))
    str_z = ",".join(list_z).replace(",", ";")
    query = {'dataset_id': 2, 'concept_id_1':'90003560', 'concept_id_2': str_z}
    info = requests.get('https://rare.cohd.io/api/frequencies/pairedConceptFreq', params=query).json()
    df = json_normalize(info['results'])
    result = pd.concat([result, df])
    i = i+100
    if (i-1) % 1000 == 0:
        print(i)
    try:
        info is None
    except:
        pass

result = result.sort_values(by=['concept_pair_count'], ascending=False)
result.head(10)

1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001


,concept_code_1,concept_code_2,concept_count_1,concept_count_2,concept_frequency,concept_id_1,concept_id_2,concept_name_1,concept_name_2,concept_pair_count,domain_id_1,domain_id_2,vocabulary_id_1,vocabulary_id_2
0,HP:0003560,HP:0025406,1611,123708,0.000620,90003560,90025406,Muscular dystrophy,Asthenia,837,phenotypes,phenotypes,hpo,hpo
0,HP:0003560,HP:0012531,1611,476026,0.000612,90003560,90012531,Muscular dystrophy,Pain,826,phenotypes,phenotypes,hpo,hpo
0,HP:0003560,HP:0003198,1611,8078,0.000450,90003560,90003198,Muscular dystrophy,Myopathy,608,phenotypes,phenotypes,hpo,hpo
0,HP:0003560,HP:0001324,1611,25288,0.000381,90003560,90001324,Muscular dystrophy,Muscle weakness,514,phenotypes,phenotypes,hpo,hpo
0,HP:0003560,HP:0012378,1611,157009,0.000379,90003560,90012378,Muscular dystrophy,Fatigue,511,phenotypes,phenotypes,hpo,hpo
0,HP:0003560,HP:0002094,1611,145671,0.000347,90003560,90002094,Muscular dystrophy,Dyspnea,469,phenotypes,phenotypes,hpo,hpo
0,HP:0003560,HP:0012735,1611,131565,0.000293,90003560,90012735,Muscular dystrophy,Cough,396,phenotypes,phenotypes,hpo,hpo
0,HP:0003560,HP:0003805,1611,4948,0.000249,90003560,90003805,Muscular dystrophy,Rimmed vacuoles,336,phenotypes,phenotypes,hpo,hpo
0,HP:0003560,HP:0006785,1611,351,0.000233,90003560,90006785,Muscular dystrophy,Limb-girdle muscular dystrophy,315,phenotypes,phenotypes,hpo,hpo
0,HP:0003560,HP:0003202,1611,13529,0.000230,90003560,90003202,Muscular dystrophy,Skeletal muscle atrophy,311,phenotypes,phenotypes,hpo,hpo


In [13]:
o_100 = o_list[16401:16480]
s = ",".join(o_100).replace(",", ";")
para = {'q': s}
response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params = para).json()
df_z = json_normalize(response['results'])
list_z = df_z['concept_id'].values.tolist()
list_z = list(map(str, list_z))
str_z = ",".join(list_z).replace(",", ";")
query = {'dataset_id': 2, 'concept_id_1':'90003560', 'concept_id_2': str_z}
info = requests.get('https://rare.cohd.io/api/frequencies/pairedConceptFreq', params=query).json()
df1 = json_normalize(info['results'])
result = pd.concat([result, df1])
result = result.loc[:,("concept_name_2", "concept_count_1", "concept_count_2", "concept_pair_count")]
result = result.sort_values(by=['concept_pair_count'], ascending=False)
result['Jaccard'] = result['concept_pair_count']/(result['concept_count_1']+result['concept_count_2']-result['concept_pair_count'])
result.head(10)


,concept_name_2,concept_count_1,concept_count_2,concept_pair_count,Jaccard
0,Asthenia,1611,123708,837,0.006724
0,Pain,1611,476026,826,0.001732
0,Myopathy,1611,8078,608,0.066953
0,Muscle weakness,1611,25288,514,0.019481
0,Fatigue,1611,157009,511,0.003232
0,Dyspnea,1611,145671,469,0.003195
0,Cough,1611,131565,396,0.002982
0,Rimmed vacuoles,1611,4948,336,0.053993
0,Limb-girdle muscular dystrophy,1611,351,315,0.191257
0,Skeletal muscle atrophy,1611,13529,311,0.020972


### Asthenia, Pain, Myopathy, Muscle weakness, Fatigue, Dyspnea, Cough,	Rimmed, Limb-girdle muscular dystrophy,	Skeletal muscle atrophy are the top 10